In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import os

In [11]:
def read_shp(file_path):
    return gpd.read_file(file_path)

def extract_endpoints(geodataframe):
    endpoints = []
    for line in geodataframe.geometry:
        if isinstance(line, LineString):
            endpoints.append(Point(line.coords[0]))
            endpoints.append(Point(line.coords[-1]))
    return gpd.GeoDataFrame(geometry=endpoints, crs=geodataframe.crs)

def shorten_line(line, distance):
    if line.length <= 2 * distance:
        return None
    start = line.interpolate(distance)
    end = line.interpolate(-distance)
    new_coords = [start] + list(line.coords[1:-1]) + [end]
    return LineString(new_coords)

def shorten_lines(geodataframe, distance):
    shortened_lines = []
    for line in geodataframe.geometry:
        if isinstance(line, LineString):
            shortened_line = shorten_line(line, distance)
            if shortened_line:
                shortened_lines.append(shortened_line)
    return gpd.GeoDataFrame(geometry=shortened_lines, crs=geodataframe.crs)

def main(input_shp, output_endpoints_shp, output_shortened_shp, distance=100):

    gdf = read_shp(input_shp)

    endpoints_gdf = extract_endpoints(gdf)
    endpoints_gdf.to_file(output_endpoints_shp)
    
    shortened_gdf = shorten_lines(gdf, distance)
    shortened_gdf.to_file(output_shortened_shp)

In [12]:
input_shp = 'cable_Test.shp' 
output_endpoints_shp = 'endpoints.shp'
output_shortened_shp = 'shortened_lines.shp'

main(input_shp, output_endpoints_shp, output_shortened_shp)